# Normalization

In a dataset, all the features (columns) may not be in same range. For eg. Price of house (thousands), Age of house (within 100) etc. It takes lot of time to train for these kind of datasets.

![BN_concept](images/image01.png)

Usually, in simpler ML algorithms like linear regression, the input is “normalized” before training to make them into single distribution. **Normalization is to convert the distribution of all inputs to have mean=0 and standard deviation=1. So more of the values lie between -1 and 1(to be specific 68.5%) and most of the values lie between -2 and +2 (to be specific 95%)**

We can even apply this normalization to the input of neural networks. It fastens up training as in linear regression. But since the first hidden layer changes this distribution, the consecutive layers are not benefited. So, what can we do? Yeah! Why not add normalization between each layers? This is what Batch normalization does.

## Batch Normalization

To reduce this problem of internal covariate shift, Batch Normalization adds Normalization “layer” between each layers. An important thing to note here is that normalization has to be done separately for each dimension (input neuron), over the ‘mini-batches’, and not altogether with all dimensions. Hence the name ‘batch’ normalization.

![batch_normalization](images/image02.png)

Due to this normalization “layers” between each fully connected layers, the range of input distribution of each layer stays the same, no matter the changes in the previous layer. Given x inputs from k-th neuron.

![formula_image](images/image03.png)

Normalization brings all the inputs centered around 0. This way, there is not much change in each layer input. So, layers in the network can learn from the back-propagation simultaneously, without waiting for the previous layer to learn. This fastens up the training of networks.

Batch Normalization is great. But, there are a few minor issues with BatchNorm which we need to take care of.

### Scale and Shift

There are usually two types in which Batch Normalization can be applied:

**1. Before activation function** (non_linear Activation function)


**2. After activation fuction**

In the [original paper](https://arxiv.org/pdf/1502.03167.pdf), BatchNorm is applied before the applying activation. Most of the activation functions have problems while applied this way. For sigmoid and tanh activation, normalized region is more of linear than nonlinear.

![Sigmoid and Tanh become linear near zero](images/image04.png)

For relu activation, half of the inputs are zeroed out.

![Relu activation](images/image05.png)

So, some transformation has to be done to move the distribution away from 0(zero mean). A scaling factor **γ** and shifting factor **β** are used to do this.

![scaling and shifting formula](images/image06.png)

As training progresses, these **γ** and **β** also learn through backpropagation so as to improve accuracy. This imposes that 2 extra parameters be learnt for each layer to increase training speed.

This final transformation thus completes definition of Batch Normalization algorithm. Use of scaling and shifting is particularly much useful because, it provides more flexibility. Suppose if we decide not to use BatchNorm, we can set **γ = σ and β = mean**, thus giving back the original values.


Recently, it has been observed that the BatchNorm when applied after activation, performs better and even gives better accuracy. For such case, we may decide to use only BatchNorm alone and not scaling and shifting. For such, set **γ = 1 and β = 0**. Nevertheless, γ and β are included in Batch Normalization Algorithm.

### BatchNorm at Inference time

We now know that Batch Normalization calculates mean and variance for each mini-batch at training time and learns using back-propagation. But, what to do at Inference time? It would not be ok if we are to use mean and variance of testing mini-batch, because it may be skewed.


What we do is, we calculate “population average” of mean and variances after training, using all the mini-batch mean and variances. And at inference time, we fix the mean and variance to be this value and use it in normalization. This provides more accurate value of mean and variance.

![population](images/population_mean_variance.png)

But, sometimes, it is difficult to keep track of all the mini-batch mean and variances. In such cases, exponential weighted “moving average” can be used to update population mean and variance:

![Moving Averages](images/moving_averages.png)

Here α is the “momentum” given to previous moving statistic, around 0.9. And those with B subscript are mini-batch mean and mini-batch variance. This is the implementation found in most libraries, where the momentum can be set manually.


An important thing to note here is that the moving mean and moving variance are calculated at training time, with training dataset and not at testing time.

### Regularization by BatchNorm

In addition to fastening up the learning of neural networks, BatchNorm also provides a weak form of regularization. How does it introduce Regularization? Regularization may be caused by introduction of noise to the data. Since the normalization is not performed on the whole dataset and just on the mini-batch, they act as noise.


However BatchNorm provides only a weak regularization, it must not be fully relied upon to avoid over-fitting. Yet, other regularization could be reduced accordingly. For example, if dropout of 0.6 (drop rate) is to be given, with BatchNorm, you can reduce the drop rate to 0.4. BatchNorm provides regularization only when the batch size is small.

### BatchNormalization in Keras

In [ ]:
keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

Batch normalization layer (Ioffe and Szegedy, 2015).

Normalize the activations of the previous layer at each batch, i.e. applies a transformation that maintains the mean activation close to 0 and the activation standard deviation close to 1.

#### Arguments

- `axis`: Integer, the axis that should be normalized (typically the features axis). For instance, after a Conv2D layer with data_format="channels_first", set axis=1 in BatchNormalization.
- `momentum`: Momentum for the moving mean and the moving variance.
- `epsilon`: Small float added to variance to avoid dividing by zero.
- `center`: If True, add offset of beta to normalized tensor. If False, beta is ignored.
- `scale`: If True, multiply by gamma. If False, gamma is not used. When the next layer is linear (also e.g. nn.relu), this can be disabled since the scaling will be done by the next layer.
- `beta_initializer`: Initializer for the beta weight.
- `gamma_initializer`: Initializer for the gamma weight.
- `moving_mean_initializer`: Initializer for the moving mean.
- `moving_variance_initializer`: Initializer for the moving variance.
- `beta_regularizer`: Optional regularizer for the beta weight.
- `gamma_regularizer`: Optional regularizer for the gamma weight.
- `beta_constraint`: Optional constraint for the beta weight.
- `gamma_constraint`: Optional constraint for the gamma weight.

#### Input shape

Arbitrary.  **Note:** Use the keyword argument `input_shape` (tuple of integers, does not include the samples axis) when using this layer as the first layer in a model.

#### Output shape

Same shape as input.

#### References

[Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift](https://arxiv.org/abs/1502.03167)
